<a href="https://colab.research.google.com/github/GarzonDiegoINL/Characterization/blob/main/EDX_PFD_files_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**EDX files reader**

In order to get the data from different EDX files, you need to upload the respective `.pdf` files, **run all** the cells and later **check the results** and download the `EDX results.xlsx` file that will have all the values.

**Note:** Please remember that this only considers the Atomic percentages values.

In [ ]:
pip install pypdf

In [ ]:
from pypdf import PdfReader
import glob as gl
import pandas as pd
import re

In [ ]:
files = gl.glob('*.pdf')
files

['240826_CIGSe_ICP_center_20kV_1.pdf',
 '240826_CIGSe_ICP_corner 4_20kV_1000x.pdf',
 '240826_CIGSe_ICP_center_20kV_1000x.pdf',
 '240826_CIGSe_ICP_center_20kV_4.pdf',
 '240826_CIGSe_ICP_Se corner_20kV_1000x.pdf',
 '240826_CIGSe_ICP_center_20kV_2.pdf',
 '240826_CIGSe_ICP_corner 3_20kV_1000x.pdf',
 '240826_CIGSe_ICP_center_20kV_3.pdf',
 '240826_CIGSe_ICP_corner 2_20kV_1000x.pdf']

In [ ]:
dfs = []
Ga = 0
for filename in files:
    reader = PdfReader(filename)
    page = reader.pages[0]
    text = page.extract_text()
    atomic_percentages = {}
    matches = re.findall(r'([A-Za-z]{1,2})\s+[A-Za-z]\s+[\d\.]+\s+([\d\.]+)', text)
    Ga = 0
    Cu = None
    In = None
    Se = 0
    for mat in matches:
        element = mat[0]
        atomic_percentage = float(mat[1])
        atomic_percentages['File'] = filename.split('.pdf')[0]
        atomic_percentages[element] = atomic_percentage
        match element:
          case 'Cu':
            Cu = float(mat[1])
          case 'In':
            In = float(mat[1])
          case 'Ga':
            Ga = float(mat[1])
          case 'Se':
            Se = float(mat[1])
    if Cu and In is not None:
        atomic_percentages['CGI'] = Cu/(Ga+In)
        atomic_percentages['GGI'] = Ga/(Ga+In)
    if Se is not None:
        atomic_percentages['SCGI'] = Se/(Cu+Ga+In)
    df = pd.DataFrame([atomic_percentages])
    dfs.append(df)
dfEDX = pd.concat(dfs, ignore_index=True)
dfEDX.to_excel('EDX results.xlsx')
display(dfEDX)

,File,Cu,Ga,Se,Mo,In,CGI,GGI,SCGI
0,240826_CIGSe_ICP_center_20kV_1,26.57,7.81,46.11,3.22,16.30,1.102032,0.323932,0.909826
1,240826_CIGSe_ICP_corner 4_20kV_1000x,22.99,6.79,47.53,5.89,16.80,0.974565,0.287834,1.020395
2,240826_CIGSe_ICP_center_20kV_1000x,26.15,6.13,51.05,NaN,16.67,1.146930,0.268860,1.042901
3,240826_CIGSe_ICP_center_20kV_4,24.12,5.30,52.05,NaN,18.53,1.012170,0.222409,1.085506
4,240826_CIGSe_ICP_Se corner_20kV_1000x,20.74,5.84,52.02,NaN,21.39,0.761660,0.214469,1.084428
5,240826_CIGSe_ICP_center_20kV_2,23.43,5.55,54.41,NaN,16.61,1.057310,0.250451,1.193463
6,240826_CIGSe_ICP_corner 3_20kV_1000x,23.10,6.76,37.56,18.15,14.44,1.089623,0.318868,0.847856
7,240826_CIGSe_ICP_center_20kV_3,22.84,7.73,51.10,NaN,18.34,0.876103,0.296509,1.044776
8,240826_CIGSe_ICP_corner 2_20kV_1000x,24.49,7.51,47.09,6.49,14.43,1.116226,0.342297,1.014215
